# **빅데이터의 이해 - 실습 (2022년 6월 8일)**

대학원 기술경영학과 (Management of Technology) - 송지훈 교수

**독학(self-study) 및 실습을 위한 팁**\
✅ You can only learn data science by doing data science. (실제로 코드를 구현해 봐야 합니다.) \
✅ Practice, practice, practice. (연습하고 또 연습하세요, 강의만으로는 모든 세세한 내용을 전부 다룰수 없습니다.)\
✅ Free resources everywhere. (인터넷상에는 수많은 무료 데이터 분석 또는 프로그래밍 관련 공부를 할 수 있는 많은 자료들이 존재 합니다. 적극적으로 찾아서 이용하세요.)

**구글코랩**: 브라우저에서 python 코드를 작성하고 실행을 가능하게 해주는 클라우드 기반의 jupyter notebook.\
자세한 내용은 다음 사이트 참조: https://colab.research.google.com/?hl=ko#scrollTo=P-H6Lw1vyNNd

# **프로젝트1: 이미지 분류 어플리케이션 (image classifier application) 생성 및 배포하기**

🔺 (입력 데이터: 이미지) ▶▶ (모델/ 앱) ▶▶ (출력 데이터: 분류된 이미지 - 라벨링)

🔺 다음의 과정을 거쳐 image classifier application 생성 및 배포

1.   gradio 라이브러리 설치 (라이브러리:특정 기능들(함수, 변수, 클래쓰)이 구현되어는 파이썬 파일 )
2.   분류하고자 하는 이미지 다운받기 (또는 불러오기)
3.   필요한 라이브러리 불러오기
4.   모델 생성하기
5.   웹 어플리케이션 생성 및 배포
6.   웹 어플리케이션 사용




참고자료: https://www.tensorflow.org/tutorials/images/transfer_learning?hl=ko

## 1.필수 라이브러리 설치

In [ ]:
# gradio 라이브러리 설치
# 분류 모델을 웹으로 배포하기 위해 필요한 라이브러리
!pip install -q gradio

## 2.이미지 불러오기 또는 웹에서 다운받기

In [ ]:
# 이미지는 미리 구글 드라이브에 저장

## 3.필요한 라이브러리 불러오기

In [ ]:
# 필요한 라이브러리 불러오기
import gradio as gr
import tensorflow as tf # 딥러닝 위한 라이브러리
import numpy as np # 수학적 계산을 위한 라이브러리 + 이미지 처리를 위해 사용
import cv2 # 이미지 처리를 위한 라이브러리
import matplotlib.pyplot as plt # 이미지/그림 시각화를 위한 라이브러리
from keras.applications.mobilenet_v2 import preprocess_input # 딥러닝 모델(mobilenetv2)을 위한 데이터 전처리 과정
from keras.applications.mobilenet_v2 import decode_predictions # 딥러닝 모델(mobilenetv2) 해석을 위한 과정
import requests # 웹사이트를 불러오기 위한 라이브러리

## 3-1. 이미지 불러오기

In [ ]:
# 이미지 불러오기
img_basic = cv2.imread('/content/drive/MyDrive/images/chippy1.jpg', cv2.IMREAD_COLOR) 
print(img_basic.shape) # 세로, 가로, RGB 색

In [ ]:
print(img_basic)

In [ ]:
plt.imshow(cv2.cvtColor(img_basic, cv2.COLOR_BGR2RGB)) # 일반적으로 이미지는 RGB (Red-Green-Blue) 형태로 저장이 되지만, cv2를 이용하면 BGR
plt.show()

In [ ]:
# 흑백으로 바꾸기 
그림흑백 = cv2.cvtColor(img_basic ,cv2.COLOR_BGR2GRAY)
plt.imshow(그림흑백, cmap='gray') # 
plt.show()

## 3-2. 이미지 크기 조절

In [ ]:
# 이미지 크기 조절
# 크기를 조정해야 하는 이유: 모델을 학습하기 위해서는 특정한 크기의 이미지가 필요
# dsize = 가로 세로 형태 조정
# interploation = 변경된 이미지 픽셀 사이의 값을 결정
다람쥐1 = cv2.imread('/content/drive/MyDrive/images/chippy1.jpg')
res1 = cv2.resize(다람쥐1, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)
res2 = cv2.resize(다람쥐1, dsize=(224, 224), interpolation=cv2.INTER_AREA) # 방법이 조금더 자연스러워 보임
plt.subplot(1, 2, 1) # 1행 2열에서 1번째 열
plt.imshow(cv2.cvtColor(res1, cv2.COLOR_BGR2RGB))
plt.subplot(1, 2, 2) # 1행 2열에서 2번째 열
plt.imshow(cv2.cvtColor(res2, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
print(res2.shape)

In [ ]:
data = np.empty((1,224,224,3)) # 숫자 1의 의미: 하나의 이미지만 사용해서 모델에, 224(이미지의 높이, 세로), 224(이미지의 너비, 가로), 3:RGB color
data[0] = res2

In [ ]:
data

In [ ]:
print(data.min(), data.max())

In [ ]:
data_new = preprocess_input(data) # 변환 작업이 필요, 모델은 [-1, 1]의 픽셀 값을 예상
data_new

## 4.모델 생성하기 + 이미지 분류 하기


In [ ]:
# MobileNetV2는 사전에 학습된 신경망 (neural network) 모델을 활용해 이미지 분류를 수행
# (pre-trained neural network model)
# MobileNetV2: 모바일 기기에 활용되기 적합한 모델로 상대적으로 "가볍다"
# model = tf.keras.applications.mobilenet_v2.MobileNetV2(weights='imagenet')
model = tf.keras.applications.MobileNetV2()

In [ ]:
model.summary()

In [ ]:
predictions = model.predict(data_new)

In [ ]:
predictions

In [ ]:
predictions.shape # 1의 의미: 1개의 이미지를 활용해 예측, 1000: 1000개의 뉴런이 output layer에 존재

In [ ]:
np.argmax(predictions, axis =1) # 몇 번째 인덱스에 제일 높은 값이 들어있는지 반환

In [ ]:
predictions[0][335]

In [ ]:
for name, desc, score in decode_predictions(predictions, top=5)[0]:
  print(f"{desc}: {score*100}%")

## 4.1 다른 이미지로 테스트

In [ ]:
# 치타 이미지 불러오기
img_basic = cv2.imread('/content/drive/MyDrive/images/cheetah.jpg', cv2.IMREAD_COLOR) 
plt.imshow(cv2.cvtColor(img_basic, cv2.COLOR_BGR2RGB)) # 
plt.show()

In [ ]:
# 이미지 크기 조절
# 크기를 조정해야 하는 이유: 모델을 학습하기 위해서는 특정한 크기의 이미지가 필요
# dsize = 가로 세로 형태 조정
# interploation = 변경된 이미지 픽셀 사이의 값을 결정
치타 = cv2.imread('/content/drive/MyDrive/images/cheetah.jpg')
res1 = cv2.resize(치타, dsize=(224, 224), interpolation=cv2.INTER_CUBIC) # 방법이 조금더 자연스러워 보임
res2 = cv2.resize(치타, dsize=(224, 224), interpolation=cv2.INTER_AREA) 
plt.subplot(1, 2, 1) # 1행 2열에서 1번째 열
plt.imshow(cv2.cvtColor(res1, cv2.COLOR_BGR2RGB))
plt.subplot(1, 2, 2) # 1행 2열에서 2번째 열
plt.imshow(cv2.cvtColor(res2, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
data = np.empty((1,224,224,3)) 
data[0] = res1
data_new = preprocess_input(data)
model = tf.keras.applications.MobileNetV2(weights='imagenet')
predictions = model.predict(data_new)
for name, desc, score in decode_predictions(predictions, top=5)[0]:
  print(f"{desc}: {score*100}%")

## 4.2 다른 이미지로 테스트

In [ ]:
# 축구공 이미지 불러오기
img_basic = cv2.imread('/content/drive/MyDrive/images/ball.jpg', cv2.IMREAD_COLOR) 
plt.imshow(cv2.cvtColor(img_basic, cv2.COLOR_BGR2RGB)) # 
plt.show()

In [ ]:
# 이미지 크기 조절
# 크기를 조정해야 하는 이유: 모델을 학습하기 위해서는 특정한 크기의 이미지가 필요
# dsize = 가로 세로 형태 조정
# interploation = 변경된 이미지 픽셀 사이의 값을 결정
축구공 = cv2.imread('/content/drive/MyDrive/images/ball.jpg')
res1 = cv2.resize(축구공, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)
res2 = cv2.resize(축구공, dsize=(224, 224), interpolation=cv2.INTER_AREA) # 방법이 조금더 자연스러워 보임
plt.subplot(1, 2, 1) # 1행 2열에서 1번째 열
plt.imshow(cv2.cvtColor(res1, cv2.COLOR_BGR2RGB))
plt.subplot(1, 2, 2) # 1행 2열에서 2번째 열
plt.imshow(cv2.cvtColor(res2, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
data = np.empty((1,224,224,3)) 
data[0] = res2
data_new = preprocess_input(data)
model = tf.keras.applications.MobileNetV2(weights='imagenet')
predictions = model.predict(data_new)
for name, desc, score in decode_predictions(predictions, top=5)[0]:
  print(f"{desc}: {score*100}%")

## 4.3 이미지 분류 과정 함수화

상대적으로 분류기의 성능이 떨어지는 느낌. 이유는 이미지 크기를 조절함에 있어서, 비율조정에 문제(?).

In [ ]:
response = requests.get("https://git.io/JJkYN")
labels = response.text.split("\n")
print(type(labels))
print(labels[:10]) # 이미지의 레이블들을 저장 (미리 학습된 내용)

In [ ]:
def predict_image(img):
  img_input=img.reshape(1,224,224,3) # 이미지를 입력받아 사이즈를 재조정
  data_new = preprocess_input(img_input)
  predictions = model.predict(data_new).flatten()
  return {labels[i]: float(predictions[i]) for i in range(1000)}

## 5.웹 어플리케이션 생성 및 배포

In [ ]:
iface = gr.Interface(
    predict_image, 
    gr.inputs.Image(shape=(224, 224)), 
    gr.outputs.Label(num_top_classes=5),
    interpretation="default",
   examples=[
        ["/content/drive/MyDrive/images/dog.jpg"],
        ["/content/drive/MyDrive/images/cheetah.jpg"],
        ["/content/drive/MyDrive/images/ball.jpg"]
    ])

## 6.웹 어플리케이션 사용

In [ ]:
iface.launch(debug=True)

In [ ]:
iface.close()

## **추가설명**

In [ ]:
# 추가설명
from keras.preprocessing import image
test_image = image.load_img("/content/drive/MyDrive/images/chippy1.jpg", target_size = (224,224))
test_image = image.img_to_array(test_image)
test_image

In [ ]:
predictions = predict_image(test_image)
predictions

In [ ]:
import pandas as pd
df = pd.DataFrame(list(predictions.items()), columns =['Label','scores'])
df.head()

In [ ]:
df['scores'] = df['scores'].round(decimals = 4)

In [ ]:
df.sort_values(by=['scores'], ascending=False)

# **프로젝트2: 타이타닉 생존자 예측하기 (생존자를 예측하는 머신러닝 모델 구현)**

지도학습의 일종으로, "binary classification" 문제

https://dacon.io/ (한국어)

https://www.kaggle.com/ (영어)

**데이터 분석을 잘 하는 방법**: "구글링(구글 검색)을 잘하기" + "다른 사람의 코드를 보고 학습"

## 1.라이브러리 불러오기

In [ ]:
import pandas as pd # 엑셀과 같은 기능을 구현해주는 데이터 사이언스를 위한 필수 라이브러리
import numpy as np
import matplotlib.pyplot as plt # 시각화 라이브러리
import seaborn as sns # 시각화 라이브러리
import gradio as gr
import warnings # warning 문구가 나오는걸 방지하기 위함
warnings.filterwarnings("ignore")

## 2.데이터 불러오기

In [ ]:
# 웹 공간에서 직접 데이터를 불러옴
data = pd.read_csv('https://raw.githubusercontent.com/gradio-app/titanic/master/train.csv') 

In [ ]:
data.shape # 데이터의 형태를 나타냄 (행과 열): 총 891 행과 12개의 열로 이루어진 데이터

## 3.데이터 탐색(EDA, Exploratory Data Analysis)

In [ ]:
data.head() # 데이터의 앞 부분 첫 5개의 행을 출력

* PassengerId : 탑승객의 고유 아이디
* Survied : 생존여부(0: 사망, 1: 생존)
* Pclass : 등실의 등급(1: 1등석, 2: 2등석, 3: 3등석)
* Name : 이름
* Sex : 성별
* Age : 나이
* Sibsp : 함께 탑승한 형제자매, 아내 남편의 수
* Parch: 함께 탑승한 부모, 자식의 수
* Ticket: 티켓번호
* Fare: 티켓의 요금
* Cabin: 객실번호
* Embarked: 배에 탑승한 위치(C = Cherbourg, Q = Queenstown, S = Southampton)

In [ ]:
data.info() # 데이터 프레임의 기본 정보를 출력

In [ ]:
data.isnull().sum() # 결측치 확인

**나이, 객실번호, 탑승 위치 에 데이터가 부재**

In [ ]:
# 생존자 예측의 문제
data.groupby("Survived").size().reset_index(name = 'count')
# 0: 사망
# 1: 생존

In [ ]:
f,ax = plt.subplots(figsize=(8,8)) # 그림의 사이즈 조정 (figsize)
ax = sns.countplot('Survived',data=data,palette='mako') # palette를 용해 색깔 조정
# 그래프들은 훨씬 더 깔끔하게 작성이 가능
# 추가적인 파라미터들이 필요 (지금은 생략)

In [ ]:
f,ax = plt.subplots(figsize=(10,8))
ax = sns.countplot('Survived',data=data,palette='mako', hue='Sex') 

In [ ]:
data.groupby(['Sex','Survived'])['Survived'].count().to_frame()

In [ ]:
f,ax = plt.subplots(figsize=(10,8))
ax = sns.countplot('Pclass',data=data,palette='mako', hue='Sex')

In [ ]:
plt.figure(figsize=(10,8))
sns.factorplot('Pclass','Survived',hue='Sex',data=data, palette='mako')

In [ ]:
pd.crosstab([data['Sex'],data['Survived']],data['Pclass'],margins=True).style.background_gradient(cmap='Reds')
# Pclass의 경우, 순서가 있는 카테고리 (ordinal features)

In [ ]:
# 탑승지역: S지역 (Southampton)
# 범주형 데이터로 처리 (categorical data)
f,ax = plt.subplots(1,3,figsize=(20,8))
sns.countplot('Embarked',data=data,palette='mako', hue='Sex', ax=ax[0])
ax[0].set_title('Embarked and Sex')
sns.countplot('Embarked',data=data,palette='mako', hue='Survived', ax=ax[1])
ax[1].set_title('Embarked and Survived')
sns.countplot('Embarked',data=data,palette='mako', hue='Pclass', ax=ax[2])
ax[2].set_title('Embarked and Pclass')

In [ ]:
pd.crosstab([data['Sex'],data['Survived'],data['Embarked']],data['Pclass'],margins=True).style.background_gradient(cmap='Reds')
# Pclass의 경우, 순서가 있는 카테고리 (ordinal features)

In [ ]:
data[data['Embarked'].isnull()]

In [ ]:
print('제일 나이가 많은 탑승객:',data['Age'].max(),'세')
print('제일 나이가 얼니 탑승객:',data['Age'].min(),'세')
print('탑승객들의 평균 나이:',data['Age'].mean(),'세')

In [ ]:
# 히스토그램으로 표현 (나이의 분포도를 나타냄)
plt.figure(figsize=(12,6))
sns.histplot(x='Age',data=data, hue='Sex')

In [ ]:
print('제일 비싼 티켓의 가격:',round(data['Fare'].max(),2))
print('제일 값싼 티켓의 가격:',round(data['Fare'].min(),2))
print('티켓의 평균 가격:',round(data['Fare'].mean(),2))

In [ ]:
# 히스토그램으로 표현 (티켓가격의 분포도를 나타냄)
plt.figure(figsize=(12,6))
sns.histplot(x='Fare',data=data, hue='Sex')

## 4.결측값 핸들링

In [ ]:
# 탑승 위치
data['Embarked'].fillna('S',inplace=True) # inplace=True 바로 결측값을 S로 채움.

In [ ]:
data.isnull().sum()

In [ ]:
# 나이 
data['Name'].head(20)

In [ ]:
data['Initial'] = data['Name'].str.extract('([A-Za-z]+)\.')
data['Initial'].value_counts()

In [ ]:
# Mlle 미혼 여성을 지칭
# Countess 백작 부인 등 - 호칭에 의해 
# Jonkheer 귀족 층 
data['Initial'] = data["Initial"].replace(["Mlle", "Mme", "Ms", "Dr", "Major", "Lady", "Countess", "Jonkheer","Col", "Rev", "Capt", "Sir", "Don"],
                             ["Miss", "Miss", "Miss", "Mr", "Mr", "Mrs", "Mrs", "Other", "Other", "Other", "Mr", "Mr", "Mr"])

In [ ]:
data.groupby('Initial')['Age'].mean()

In [ ]:
data.groupby('Initial')['Age'].median()

In [ ]:
# 결측치를 이니셜/타이틀에 맞게 핸들링
# 평균을 기준으로 결측치를 보완
data.loc[(data['Age'].isnull())&(data['Initial'] =='Mr'),'Age'] = 33
data.loc[(data['Age'].isnull())&(data['Initial']=='Mrs'),'Age'] = 36
data.loc[(data['Age'].isnull())&(data['Initial']=='Master'),'Age'] = 5
data.loc[(data['Age'].isnull())&(data['Initial']=='Miss'),'Age'] = 22
data.loc[(data['Age'].isnull())&(data['Initial']=='Other'),'Age'] = 46

In [ ]:
data.isnull().sum()

In [ ]:
# 히스토그램으로 표현 (나이의 분포도를 나타냄)
plt.figure(figsize=(12,6))
sns.histplot(x='Age',data=data, hue='Survived')

In [ ]:
plt.figure(figsize=(12,6))
sns.kdeplot(x='Age',data=data, hue='Survived')

In [ ]:
# 객실 결측치
# 따로 처리하지 않고 삭제 후 진행

## 5.피처 엔지니어링

모델생성에 필요한 변수들을 추출 및 알맞게 변환하는 과정

In [ ]:
data.head()

In [ ]:
gender_mapping = {"male": 0, "female": 1}
data['Gender'] = data['Sex'].map(gender_mapping)

In [ ]:
# 요금
plt.figure(figsize=(12,6))
sns.kdeplot(x='Fare',data=data, hue='Survived')

In [ ]:
pd.qcut(data['Fare'],4).unique()

In [ ]:
data['Fare_cat']=0
data.loc[data['Fare']<=7.91,'Fare_cat'] = 0
data.loc[(data['Fare']>7.91)&(data['Fare']<=14.454),'Fare_cat'] = 1
data.loc[(data['Fare']>14.454)&(data['Fare']<=31),'Fare_cat'] = 2
data.loc[(data['Fare']>31)&(data['Fare']<=513),'Fare_cat'] = 3

In [ ]:
data.head()

In [ ]:
data['Age'].median(), data['Age'].max()

In [ ]:
data['Fare'].median(), data['Fare'].max()

In [ ]:
# 불필요한 컬럼 제거 (필요한 열만 추출해서 사용)
data_selected = data[['Survived','Pclass','Age','SibSp','Parch','Fare_cat','Embarked','Initial','Gender']]
data_selected.head()

In [ ]:
correlation = data_selected.corr()
correlation

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(correlation,annot=True,cmap='Reds',linewidths=0.2)

In [ ]:
pd.qcut(data_selected['Age'],5).unique() # qcut: 분위수 기반으로 데이터를 나누는 과정. 

In [ ]:
pd.cut(data_selected['Age'],5).unique() # cut: 데이터를 동일한 길이로 나누는 과정

In [ ]:
data_selected['Age_band'] = pd.cut(data_selected['Age'], 5, labels=[0,1,2,3,4])
data_selected['Age_band'] = data_selected['Age_band'].astype(int)

In [ ]:
data_selected.head()

In [ ]:
# data_selected['Age_band'] = 0
# data_selected.loc[data_selected['Age']<=16,'Age_band'] = 0
# data_selected.loc[(data_selected['Age']>16)&(data_selected['Age']<=32),'Age_band'] = 1
# data_selected.loc[(data_selected['Age']>32)&(data_selected['Age']<=48),'Age_band'] = 2
# data_selected.loc[(data_selected['Age']>48)&(data_selected['Age']<=64),'Age_band'] = 3
# data_selected.loc[data_selected['Age']>64,'Age_band'] = 4

In [ ]:
correlation2 = data_selected.corr()
plt.figure(figsize=(10,6))
sns.heatmap(correlation2,annot=True,cmap='Reds',linewidths=0.2)

In [ ]:
data_selected['FamilySize'] = 0
data_selected["FamilySize"] = data_selected["SibSp"] + data_selected["Parch"]
data_selected['Alone'] = 0
data_selected.loc[data_selected["FamilySize"]==0,'Alone'] = 1 

In [ ]:
data_selected.isnull().sum()

In [ ]:
plt.figure(figsize=(12,6))
sns.kdeplot(x='FamilySize',data=data_selected, hue='Survived')

In [ ]:
data_selected["FamilySize"].unique()

In [ ]:
data_selected.head()

In [ ]:
# 불필요한 부분 (나중에 바꿔서 테스트 해볼 수 있는 부분)
# family_mapping = {1: 0, 2: 0.4, 3: 0.8, 4: 1.2, 5: 1.6, 6: 2, 7: 2.4, 8: 2.8, 9: 3.2, 10: 3.6, 11: 4}
# data_selected['FamilySize'] = data_selected['FamilySize'].map(family_mapping)

In [ ]:
data_selected = data_selected.drop(['SibSp','Parch','Age'], axis=1)
data_selected.head()

In [ ]:
#X = data_selected.drop('Survived', axis=1).copy()
#y = data_selected[['Survived']].copy()

In [ ]:
X = data_selected.copy()
X

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
ohe = OneHotEncoder()

In [ ]:
# ct = make_column_transformer(
#     (ohe, ['Embarked', 'Initial']), remainder='passthrough') 
# ct.fit_transform(X)
# pd.DataFrame(ohe.fit_transform(X[['Embarked', 'Initial']]).toarray())

In [ ]:
data_input = pd.get_dummies(data=X, columns=['Embarked', 'Initial'])
data_input

In [ ]:
X = data_input.iloc[:, 1:]  # feature
y = data_input.iloc[:, 0]   # label

## 6.모델 구현하기 + 최적의 모델 선정 (scikit-learn 사용)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2022, stratify=data_input["Survived"])
# stratify =  훈련데이터를 나눌 때 무작위로 샘플링을 하되, original dataset의 클래스 비율이 train, test set에서도 동일하게 유지

In [ ]:
display(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

하나의 간단한 모델 구현

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, StratifiedKFold, GridSearchCV
from sklearn import metrics 
logreg = LogisticRegression()
# k_fold = KFold(n_splits=5, shuffle=True, random_state=2022)
k_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2020)
score = cross_val_score(logreg, X_train, y_train, cv=k_fold, scoring='accuracy')

In [ ]:
score

In [ ]:
print(score.mean())

In [ ]:
model = LogisticRegression()
model.fit(X_train,y_train)
prediction_lr = model.predict(X_test)
print('The accuracy of the Logistic Regression is', metrics.accuracy_score(prediction_lr,y_test))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [ ]:
mean_cv =[]
accuracy =[]
std=[]
classifiers=['Logistic Regression','Support Vector Machine','KNN','Decision Tree','Naive Bayes','Random Forest']
models=[LogisticRegression(),SVC(kernel='rbf'),KNeighborsClassifier(n_neighbors=10),DecisionTreeClassifier(),GaussianNB(),RandomForestClassifier(n_estimators=100)] # n_estimators = 모델에서 사용할 트리 갯수
for i in models:
    model = i
    cv_result = cross_val_score(model,X_train,y_train, cv = k_fold,scoring = "accuracy")
    mean_cv.append(cv_result.mean())
    std.append(cv_result.std())
    accuracy.append(cv_result)
models_dataframe =pd.DataFrame({'CV Mean':mean_cv,'Std':std},index=classifiers)       
models_dataframe

In [ ]:
model = SVC(kernel='rbf')
model.fit(X_train,y_train)
prediction_svc = model.predict(X_test)
print('The accuracy of the Support Vector Machine is', metrics.accuracy_score(prediction_svc,y_test))

SVM(support vector machine)의 성능을 개선

https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

In [ ]:
svr_params = {'kernel': ['rbf'],
              'gamma': [0.001, 0.01, 0.1, 1],
              'C': [1, 10, 50, 100, 200, 300, 1000]}
svr_gridsearch = GridSearchCV(SVC(),
                              svr_params,
                              cv=k_fold, scoring='accuracy',n_jobs=-1)
svr_gridsearch.fit(X_train, y_train)

In [ ]:
svr_gridsearch.best_score_

In [ ]:
svr_gridsearch.best_estimator_

In [ ]:
prediction_svr = svr_gridsearch.best_estimator_.predict(X_test)
metrics.accuracy_score(y_test, prediction_svr)

In [ ]:
# 시간이 조금 소요되는 과정
rf_params = {'max_depth': [None],
                  'max_features': [1, 3, 10],
                  'min_samples_split': [2, 3, 10],
                  'min_samples_leaf': [1, 3, 10],
                  'bootstrap': [False],
                  'n_estimators':[100,300,500],
                  'criterion': ['gini']}
rf_gridsearch = GridSearchCV(RandomForestClassifier(),
                                  rf_params,
                                  cv=k_fold,
                                  scoring='accuracy',
                                  n_jobs=-1)
rf_gridsearch.fit(X_train, y_train)

In [ ]:
rf_gridsearch.best_score_

In [ ]:
rf_gridsearch.best_estimator_

In [ ]:
y_pred_rf = rf_gridsearch.best_estimator_.predict(X_test)
metrics.accuracy_score(y_pred_rf, y_test)

In [ ]:
clf = RandomForestClassifier(bootstrap=False, max_features=1, min_samples_leaf=3,
                       min_samples_split=10, n_estimators=300)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
print(metrics.accuracy_score(y_test, predictions))

In [ ]:
features = pd.Series(
    clf.feature_importances_,
    index=X_train.columns
).sort_values(ascending=False)

print(features)

In [ ]:
top_5_features = features.keys()[:5]
print(top_5_features)

## 7.웹 어플리케이션 생성 및 배포

In [ ]:
# df = pd.DataFrame.from_dict({'Pclass': ['1'], 'FamilySize': [2], 'Alone': ['Alone'], 'Embarked': ['S'], 'Initial': ['Mr'], 'Gender': ['female'], 'Age': [6], 'SibSp': [0], 'Parch': [0], 'Fare': [18], 'Cabin': [7]})

In [ ]:
def encode_age(df):
    df['Age_band'] = 0
    df.loc[df['Age']<=16,'Age_band'] = 0
    df.loc[(df['Age']>16)&(df['Age']<=32),'Age_band'] = 1
    df.loc[(df['Age']>32)&(df['Age']<=48),'Age_band'] = 2
    df.loc[(df['Age']>48)&(df['Age']<=64),'Age_band'] = 3
    df.loc[df['Age']>64,'Age_band'] = 4
    return df

In [ ]:
def encode_fare(df):
    df['Fare_cat']=0
    df.loc[df['Fare']<=7.91,'Fare_cat'] = 0
    df.loc[(df['Fare']>7.91)&(df['Fare']<=14.454),'Fare_cat'] = 1
    df.loc[(df['Fare']>14.454)&(df['Fare']<=31),'Fare_cat'] = 2
    df.loc[(df['Fare']>31)&(df['Fare']<=513),'Fare_cat'] = 3
    return df

In [ ]:
def encode_embarked(df):
  df['Embarked_S']=0
  df['Embarked_Q']=0
  df['Embarked_C']=0
  if df['Embarked'].iloc[0] == 'S':
    df['Embarked_S'].iloc[0] = 1
  elif df['Embarked'].iloc[0] == 'Q':
    df['Embarked_Q'].iloc[0] = 1
  elif df['Embarked'].iloc[0] == 'C':
    df['Embarked_C'].iloc[0] = 1
  return df

In [ ]:
def encode_initial(df):
  df['Initial_Master']=0
  df['Initial_Miss']=0
  df['Initial_Mr']=0
  df['Initial_Mrs']=0
  df['Initial_Other']=0
  if df['Initial'].iloc[0] == 'Master':
    df['Initial_Master'].iloc[0] = 1
  elif df['Initial'].iloc[0] == 'Miss':
    df['Initial_Miss'].iloc[0] = 1
  elif df['Initial'].iloc[0] == 'Mr':
    df['Initial_Mr'].iloc[0] = 1
  elif df['Initial'].iloc[0] == 'Mrs':
    df['Initial_Mrs'].iloc[0] = 1
  elif df['Initial'].iloc[0] == 'Other':
    df['Initial_Other'].iloc[0] = 1
  return df

In [ ]:
def encode_gender(df):
    mapping = {"male": 0, "female": 1}
    return df.replace({'Gender': mapping})

In [ ]:
def encode_alone(df):
    mapping = {"With_family": 0, "Alone": 1}
    return df.replace({'Alone': mapping})

In [ ]:
def make_prediction(pclass, fare, gender, age, familysize, alone, embarked, initial):

    df = pd.DataFrame.from_dict({'Pclass': [pclass], 'Fare': [fare], 'Gender': [gender], 'Age': [age], 'FamilySize': [familysize], 
                                 'Alone': [alone], 'Embarked': [embarked], 'Initial': [initial]})
    
    df = encode_age(df)    
    df = encode_fare(df)    
    df = encode_embarked(df)   
    df = encode_initial(df)    
    df = encode_gender(df)
    df = encode_alone(df)      

    preds = clf.predict(df[['Pclass', 'Fare_cat', 'Gender', 'Age_band',	'FamilySize',	'Alone',	'Embarked_C',	'Embarked_Q',	'Embarked_S',	'Initial_Master',	'Initial_Miss',	'Initial_Mr',	'Initial_Mrs',	'Initial_Other']])
    return ['Perishes', 'Survives'][preds[0]]

In [ ]:
make_prediction(1, 80, 'female',32, 2, 0, 'S','Mrs')

In [ ]:
pclass = gr.inputs.Radio(['1','2','3'], label="Pclass")
fare = gr.inputs.Slider(minimum=0, maximum=513, default=14, label="Fare")
gender = gr.inputs.Radio(['male','female'], label="Gender")
age = gr.inputs.Slider(minimum=0, maximum=90, default=30, label="Age")
familysize = gr.inputs.Slider(minimum=0, maximum=10, default=0, label="Family size")
alone = gr.inputs.Radio(['Alone', 'With_family'], label="Alone or with family ")
embarked = gr.inputs.Radio(["S","C","Q"], label="Embarked at")
initial = gr.inputs.Radio(["Master","Miss","Mr","Mrs","Other"], label="Choose an Initial")

app = gr.Interface(make_prediction, [pclass, fare, gender, age, familysize, alone, embarked, initial],'label')
app.launch(debug=True)

In [ ]:
app.close()